In [1]:
import numpy as np
import pandas as pd
from data_mani.utils import get_market_df
from tqdm import tqdm
from glob import glob

In [48]:
path = "data/tickers/spx_group_sector.csv"
dicionario_df = pd.read_csv(path)
dicionario_df = dicionario_df.loc[dicionario_df.field == "INDUSTRY_SECTOR"].reset_index(drop=True)
dicionario_df.loc[:, "ticker"] = dicionario_df.ticker.map(lambda x: x.replace("/", " "))
sectors = dicionario_df.value.unique().tolist()
dicionario_df.rename(columns={'value': 'sectors'}, inplace=True)

In [49]:
dicionario_df.head()

,Unnamed: 0,ticker,field,sectors,exchange,stock
0,0,0111145D UN Equity,INDUSTRY_SECTOR,Utilities,UN,0111145D
1,1,0202445Q UN Equity,INDUSTRY_SECTOR,"Consumer, Non-cyclical",UN,0202445Q
2,2,0203524D UN Equity,INDUSTRY_SECTOR,Technology,UN,0203524D
3,3,0226226D UN Equity,INDUSTRY_SECTOR,"Consumer, Cyclical",UN,0226226D
4,4,0544749D UN Equity,INDUSTRY_SECTOR,Energy,UN,0544749D


In [52]:
spx_adj_ticker = glob('data/tickers/spx_adj/*.csv')

melt_out = []
for ticker in tqdm(spx_adj_ticker, desc='Build melted dataframe with stocks'):
    df = pd.read_csv(ticker)
    melt_df = df.melt('date')
    melt_out.append(melt_df)
melt_df = pd.concat(melt_out, axis=0)
melt_df.rename(columns={'variable': 'ticker'}, inplace=True)
merge_melt_df = pd.merge(melt_df, dicionario_df, on='ticker', how='left')
merge_melt_df = merge_melt_df[['date', 'ticker', 'value', 'sectors']]

Build melted dataframe with stocks: 100%|██████████| 928/928 [00:18<00:00, 49.59it/s]


In [54]:
merge_melt_df.head()

,date,ticker,value,sectors
0,2000-01-04,1284849D UN Equity,-1.1349,"Consumer, Non-cyclical"
1,2000-01-05,1284849D UN Equity,-0.3827,"Consumer, Non-cyclical"
2,2000-01-06,1284849D UN Equity,-1.7926,"Consumer, Non-cyclical"
3,2000-01-07,1284849D UN Equity,9.7784,"Consumer, Non-cyclical"
4,2000-01-08,1284849D UN Equity,NaN,"Consumer, Non-cyclical"


In [55]:
 merge_melt_df.shape[0] == melt_df.shape[0]

True

In [56]:
len(sectors) == len(merge_melt_df['sectors'].unique())

True

In [59]:
sector = merge_melt_df['sectors'].unique()[0]
complete = merge_melt_df.loc[merge_melt_df['sectors'] == sector].drop('sectors', 1).pivot_table(index=['date'], columns=['ticker'])
new_name = sector.replace(", ", " ").replace("-", " ")
sector_df = complete.mean(1).to_frame().rename(columns={0: new_name})
sector_df = sector_df
out_path = "data/indices/SPX {}.csv".format(new_name) 
sector_df.head()

,Consumer Non cyclical
date,
2000-01-04,-2.746675
2000-01-05,1.183922
2000-01-06,1.576074
2000-01-07,3.918827
2000-01-10,0.577642


In [62]:
for sector in tqdm(merge_melt_df['sectors'].unique(), desc='agg by sectors and save data'):
    sector = merge_melt_df['sectors'].unique()[0]
    complete = merge_melt_df.loc[merge_melt_df['sectors'] == sector].drop('sectors', 1).pivot_table(index=['date'], columns=['ticker'])
    new_name = sector.replace(", ", " ").replace("-", " ")
    sector_df = complete.mean(1).to_frame().rename(columns={0: new_name})
    sector_df = sector_df
    out_path = "data/indices/SPX {}.csv".format(new_name) 
    sector_df.to_csv(out_path)
    # creating the same format as the other csv's
    file_in = open(out_path, "r")
    prefix = ["ticker,SPX {}\n".format(new_name),
            "field,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS\n",
            "date,\n"] 
    lines = prefix + file_in.readlines()[1:]
    file_out = open(out_path,"w") 
    file_out.writelines(lines) 
    file_out.close()

agg by sectors and save data: 100%|██████████| 9/9 [00:23<00:00,  2.58s/it]
